In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


engine = create_engine('sqlite:///D:/python_scripts/Tasks/TODO_API/DB/TODO.db', echo=True)  # Change echo=True for logging SQL queries
Base = declarative_base()
class Todo(Base):
    __tablename__ = 'Tasks'
    id = Column(Integer, primary_key=True)
    title = Column(String(100))
    description = Column(String)
    completed = Column(Boolean, default=False)


C:\Users\nishantm\AppData\Local\Temp\ipykernel_35408\350767603.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [15]:
import redis.asyncio as redis
url_connection = redis.from_url(url="redis://redis-18552.c267.us-east-1-4.ec2.cloud.redislabs.com",port=18552,password='jdj3wE7esEVxsK9CQ1f3Z74nwBqY8wN2',decode_responses=True)
url_connection.ping()

<coroutine object Redis.execute_command at 0x0000015B54183DE0>

In [28]:
todo_keys = await url_connection.keys(f'todo:*')
todos = [
            await url_connection.hgetall(todo_key) for todo_key in todo_keys
        ]
print(todos)
print(todo_keys)

[{'id': '5', 'title': 'Task 2', 'description': 'New desc Task 5', 'completed': '1'}, {'id': '6', 'title': 'Task 3', 'description': 'Desc Task 3', 'completed': '0'}, {'id': '4', 'title': 'Task 1', 'description': 'Desc Task 1', 'completed': '0'}]
['todo:5', 'todo:6', 'todo:4']


In [29]:
url_connection.hmget("todo:5",{'id': '5', 'title': 'Task 8', 'description': 'New desc Task 5', 'completed': '1'})

<coroutine object Redis.execute_command at 0x0000015B54C649D0>

In [3]:
Session_OBJ = sessionmaker(bind=engine)
session = Session_OBJ()

In [4]:
new_todo = Todo(title="Task2", description="Task2")
session.add(new_todo)

In [5]:
from sqlalchemy import MetaData

metadata = MetaData()

# Reflect the existing database into MetaData
metadata.reflect(bind=engine)

# Drop all tables defined in the metadata
metadata.drop_all(bind=engine, checkfirst=True)

2024-01-09 11:09:36,216 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-09 11:09:36,218 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-01-09 11:09:36,219 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-09 11:09:36,221 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-01-09 11:09:36,223 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-09 11:09:36,225 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("Tasks")
2024-01-09 11:09:36,226 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-09 11:09:36,228 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2024-01-09 11:09:36,230 INFO sqlalchemy.engine.Engine [raw sql] ('Tasks',)
2024-01-09 11:09:36,232 INFO sqlalchemy.engine.Engin

In [1]:
import redis
redis_client = redis.StrictRedis(host='redis-18552.c267.us-east-1-4.ec2.cloud.redislabs.com', port=18552, decode_responses=True, password="jdj3wE7esEVxsK9CQ1f3Z74nwBqY8wN2")
  

In [2]:
todo_key_prefix = 'todo:'
import json

In [7]:
todo_keys = redis_client.keys(f'{todo_key_prefix}*')
print(todo_keys)

['todo:6', 'todo:5', 'todo:4']


In [9]:
redis_client.hgetall(todo_keys[0])

{'id': '6', 'title': 'Task 3', 'description': 'Desc Task 3', 'completed': '0'}

In [11]:
completed_todos = [
    redis_client.hgetall(todo_key) for todo_key in todo_keys
    if redis_client.hget(todo_key, 'completed') == "0"
]
print(completed_todos)

[{'id': '6', 'title': 'Task 3', 'description': 'Desc Task 3', 'completed': '0'}, {'id': '4', 'title': 'Task 1', 'description': 'Desc Task 1', 'completed': '0'}]


In [87]:
todo_keys = redis_client.keys(f'{todo_key_prefix}*')
todos = [redis_client.hgetall(todo_key) for todo_key in todo_keys]
print(todos)

ResponseError: WRONGTYPE Operation against a key holding the wrong kind of value

In [88]:
todo_key = f'{todo_key_prefix}{2}'
todo_data = redis_client.hgetall(todo_key)
if todo_data:
    print(todo_data)
query = {"title":"Task 2"}

ResponseError: WRONGTYPE Operation against a key holding the wrong kind of value

In [82]:
title_query = query.get("title", "")
description_query = query.get("description", "")
todo_keys = redis_client.keys(f'{todo_key_prefix}*')
todos = [
    redis_client.hgetall(todo_key) for todo_key in todo_keys
    if title_query.lower() in redis_client.hget(todo_key, 'title').lower()
    and description_query.lower() in redis_client.hget(todo_key,
                                                       'description').lower()
]
print(todos)

[{'id': '2', 'title': 'Task 2', 'description': 'Task Desc 2', 'completed': '0'}]


In [83]:
title = "Task New 2"
todo_id = 2
description = None
completed = None

In [84]:
todo_key = f'{todo_key_prefix}{todo_id}'
todo_data = redis_client.hgetall(todo_key)

todo = todo_data
if title:
        todo['title'] = title
if description:
        todo['description'] = description
if completed is not None:
        todo['completed'] = 0 if completed==False else 1

redis_client.set(todo_key, json.dumps(todo))

True